<h1></h1>
<h2><center> ETI195 - Ética para Ciencia de Datos y Estadística </center></h2>

<h1></h1>
<h2><center> Taller 2: Regresión Logística COMPAS, Riesgo Relativo. </center></h2>



## Imports

In [3]:
import pandas as pd
import numpy as np

import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

import statsmodels.api as sm

import seaborn as sns

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

## Recapitulación del taller pasado

<h2> COMPAS: Correctional Offender Management Profiling for Alternative Sanctions </h2>

![imagen](https://static.propublica.org/projects/algorithmic-bias/assets/img/generated/opener-b-crop-960*540-00796e.jpg)
<b><h6> Imagen 1 - Machine Bias (ProPublica) </h6></b>

- Algoritmo utilizado en el sistema de justicia criminal de Estados Unidos para predecir la probabilidad o riesgo de reincidencia de un acusado.

- Tiene por objetivo  ayudar a los jueces a tomar decisiones más informadas sobre el riesgo de reincidencia.

Investigación hecha por ProPublica: Existe una clara diferencia en la distribución de los puntajes de riesgo según raza.

<b><h3>Links de interés: </h3></b>

- [Artículo ProPublica : Machine Bias](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing)

- [Metodología](https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm)

- [Códigos originales de ProPublica](https://github.com/propublica/compas-analysis/tree/master)




In [4]:
compas = pd.read_csv(
    "https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv"
)

compas.shape

(7214, 53)

In [5]:
compas.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0


In [6]:
# Mantenemos las columnas de interés.

columns = [
    "age",
    "c_charge_degree",
    "race",
    "age_cat",
    "score_text",
    "sex",
    "priors_count",
    "days_b_screening_arrest",
    "decile_score",
    "is_recid",
    "two_year_recid",
    "c_jail_in",
    "c_jail_out",
]

compas = compas[columns]

In [7]:
compas.head()

,age,c_charge_degree,race,age_cat,score_text,sex,priors_count,days_b_screening_arrest,decile_score,is_recid,two_year_recid,c_jail_in,c_jail_out
0,69,F,Other,Greater than 45,Low,Male,0,-1.0,1,0,0,2013-08-13 06:03:42,2013-08-14 05:41:20
1,34,F,African-American,25 - 45,Low,Male,0,-1.0,3,1,1,2013-01-26 03:45:27,2013-02-05 05:36:53
2,24,F,African-American,Less than 25,Low,Male,4,-1.0,4,1,1,2013-04-13 04:58:34,2013-04-14 07:02:04
3,23,F,African-American,Less than 25,High,Male,1,NaN,8,0,0,NaN,NaN
4,43,F,Other,25 - 45,Low,Male,2,NaN,1,0,0,NaN,NaN


In [8]:
compas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7214 entries, 0 to 7213
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      7214 non-null   int64  
 1   c_charge_degree          7214 non-null   object 
 2   race                     7214 non-null   object 
 3   age_cat                  7214 non-null   object 
 4   score_text               7214 non-null   object 
 5   sex                      7214 non-null   object 
 6   priors_count             7214 non-null   int64  
 7   days_b_screening_arrest  6907 non-null   float64
 8   decile_score             7214 non-null   int64  
 9   is_recid                 7214 non-null   int64  
 10  two_year_recid           7214 non-null   int64  
 11  c_jail_in                6907 non-null   object 
 12  c_jail_out               6907 non-null   object 
dtypes: float64(1), int64(5), object(7)
memory usage: 732.8+ KB


En la metodología propuesta por ProPublica (revisar <b>Links de interés</b>) se presentan los siguientes criterios para realizar la limpieza de los datos:

- Si la fecha del cargo por el delito de un acusado evaluado por COMPAS no estaba dentro de los 30 días desde el momento en que la persona fue arrestada, asumimos que, debido a razones de calidad de datos, no tenemos el delito correcto.

- Según se indica en el código publicado por ProPublica, se etiquetó con ```is_recid = -1``` los casos para los cuales no se encontró el caso de COMPAS.

- En una línea similar, se eliminan las infracciones de tráfico comunes (aquellas con un grado de ```c_charge_degree``` <b>'O'</b>), que no resultarían en tiempo de prisión.


In [9]:
clean_df = compas[
    (
        (compas["days_b_screening_arrest"] <= 30)
        & (compas["days_b_screening_arrest"] >= -30)
        & (compas["is_recid"] != -1)
        & (compas["c_charge_degree"] != "O")
    )
]

clean_df.shape

(6172, 13)

In [10]:
compas.shape

(7214, 13)

In [11]:
print(f"Se eliminaron {compas.shape[0] - clean_df.shape[0]} registros.")

Se eliminaron 1042 regitros.


In [12]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6172 entries, 0 to 7213
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age                      6172 non-null   int64  
 1   c_charge_degree          6172 non-null   object 
 2   race                     6172 non-null   object 
 3   age_cat                  6172 non-null   object 
 4   score_text               6172 non-null   object 
 5   sex                      6172 non-null   object 
 6   priors_count             6172 non-null   int64  
 7   days_b_screening_arrest  6172 non-null   float64
 8   decile_score             6172 non-null   int64  
 9   is_recid                 6172 non-null   int64  
 10  two_year_recid           6172 non-null   int64  
 11  c_jail_in                6172 non-null   object 
 12  c_jail_out               6172 non-null   object 
dtypes: float64(1), int64(5), object(7)
memory usage: 675.1+ KB


In [13]:
age_count = clean_df["age_cat"].value_counts(normalize=True) * 100
race_count = clean_df["race"].value_counts(normalize=True) * 100
sex_count = clean_df["sex"].value_counts(normalize=True) * 100

## Pre - procesamiento (del taller previo)


In [14]:
object_columns = clean_df.select_dtypes(include="object").columns
for objcol in object_columns:
    print(f"Columna {objcol}: {clean_df[objcol].unique()}")

Columna c_charge_degree: ['F' 'M']
Columna race: ['Other' 'African-American' 'Caucasian' 'Hispanic' 'Asian'
 'Native American']
Columna age_cat: ['Greater than 45' '25 - 45' 'Less than 25']
Columna score_text: ['Low' 'Medium' 'High']
Columna sex: ['Male' 'Female']
Columna c_jail_in: ['2013-08-13 06:03:42' '2013-01-26 03:45:27' '2013-04-13 04:58:34' ...
 '2014-01-13 05:48:01' '2014-03-08 08:06:02' '2014-06-28 12:16:41']
Columna c_jail_out: ['2013-08-14 05:41:20' '2013-02-05 05:36:53' '2013-04-14 07:02:04' ...
 '2014-01-14 07:49:46' '2014-03-09 12:18:04' '2014-06-30 11:19:23']


In [15]:
object_columns = [
    x
    for x in clean_df.select_dtypes(include="object").columns
    if x != "c_jail_in" and x != "c_jail_out"
]

for objcol in object_columns:

    clean_df[objcol] = clean_df[objcol].astype("category")

In [16]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6172 entries, 0 to 7213
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   age                      6172 non-null   int64   
 1   c_charge_degree          6172 non-null   category
 2   race                     6172 non-null   category
 3   age_cat                  6172 non-null   category
 4   score_text               6172 non-null   category
 5   sex                      6172 non-null   category
 6   priors_count             6172 non-null   int64   
 7   days_b_screening_arrest  6172 non-null   float64 
 8   decile_score             6172 non-null   int64   
 9   is_recid                 6172 non-null   int64   
 10  two_year_recid           6172 non-null   int64   
 11  c_jail_in                6172 non-null   object  
 12  c_jail_out               6172 non-null   object  
dtypes: category(5), float64(1), int64(5), object(2)
memory usage: 464.8+

In [17]:
clean_df.head()

,age,c_charge_degree,race,age_cat,score_text,sex,priors_count,days_b_screening_arrest,decile_score,is_recid,two_year_recid,c_jail_in,c_jail_out
0,69,F,Other,Greater than 45,Low,Male,0,-1.0,1,0,0,2013-08-13 06:03:42,2013-08-14 05:41:20
1,34,F,African-American,25 - 45,Low,Male,0,-1.0,3,1,1,2013-01-26 03:45:27,2013-02-05 05:36:53
2,24,F,African-American,Less than 25,Low,Male,4,-1.0,4,1,1,2013-04-13 04:58:34,2013-04-14 07:02:04
5,44,M,Other,25 - 45,Low,Male,0,0.0,1,0,0,2013-11-30 04:50:18,2013-12-01 12:28:56
6,41,F,Caucasian,25 - 45,Medium,Male,14,-1.0,6,1,1,2014-02-18 05:08:24,2014-02-24 12:18:30


In [18]:
pd.get_dummies(clean_df["race"]).head()

,African-American,Asian,Caucasian,Hispanic,Native American,Other
0,False,False,False,False,False,True
1,True,False,False,False,False,False
2,True,False,False,False,False,False
5,False,False,False,False,False,True
6,False,False,True,False,False,False


In [19]:
post_df = clean_df.copy()

In [20]:
dummies = pd.get_dummies(post_df["race"])
post_df = pd.concat([post_df, dummies], axis=1)
post_df.drop(columns=["race", "Caucasian"], inplace=True)

In [21]:
clean_df.head()

,age,c_charge_degree,race,age_cat,score_text,sex,priors_count,days_b_screening_arrest,decile_score,is_recid,two_year_recid,c_jail_in,c_jail_out
0,69,F,Other,Greater than 45,Low,Male,0,-1.0,1,0,0,2013-08-13 06:03:42,2013-08-14 05:41:20
1,34,F,African-American,25 - 45,Low,Male,0,-1.0,3,1,1,2013-01-26 03:45:27,2013-02-05 05:36:53
2,24,F,African-American,Less than 25,Low,Male,4,-1.0,4,1,1,2013-04-13 04:58:34,2013-04-14 07:02:04
5,44,M,Other,25 - 45,Low,Male,0,0.0,1,0,0,2013-11-30 04:50:18,2013-12-01 12:28:56
6,41,F,Caucasian,25 - 45,Medium,Male,14,-1.0,6,1,1,2014-02-18 05:08:24,2014-02-24 12:18:30


In [22]:
dummies = pd.get_dummies(post_df["age_cat"])
post_df = pd.concat([post_df, dummies], axis=1)
post_df.drop(columns=["age_cat", "25 - 45"], inplace=True)

In [23]:
dummies = pd.get_dummies(post_df["sex"])
post_df = pd.concat([post_df, dummies], axis=1)
post_df.drop(columns=["sex", "Male"], inplace=True)

In [24]:
dummies = pd.get_dummies(post_df["c_charge_degree"])
post_df = pd.concat([post_df, dummies], axis=1)
post_df.drop(columns=["c_charge_degree", "F"], inplace=True)

## Pre - procesamiento nuevo

En el presente taller continuaremos con el análisis de los datos de ProPublica respecto del algoritmo COMPAS, que estuvimos viendo durante los tutoriales anteriores. En particular, el objetivo de este tutorial es evaluar y cuantificar los efectos de la raza y edad sobre las predicciones de este algoritmo.

Para iniciar, haremos algunas modificaciones pendientes a los datos que teníamos en el tutorial
anterior.

Recordemos como se veían estos datos:

In [26]:
# Datos taller anterior.
post_df.head()

,age,score_text,priors_count,days_b_screening_arrest,decile_score,is_recid,two_year_recid,c_jail_in,c_jail_out,African-American,Asian,Hispanic,Native American,Other,Greater than 45,Less than 25,Female,M
0,69,Low,0,-1.0,1,0,0,2013-08-13 06:03:42,2013-08-14 05:41:20,False,False,False,False,True,True,False,False,False
1,34,Low,0,-1.0,3,1,1,2013-01-26 03:45:27,2013-02-05 05:36:53,True,False,False,False,False,False,False,False,False
2,24,Low,4,-1.0,4,1,1,2013-04-13 04:58:34,2013-04-14 07:02:04,True,False,False,False,False,False,True,False,False
5,44,Low,0,0.0,1,0,0,2013-11-30 04:50:18,2013-12-01 12:28:56,False,False,False,False,True,False,False,False,True
6,41,Medium,14,-1.0,6,1,1,2014-02-18 05:08:24,2014-02-24 12:18:30,False,False,False,False,False,False,False,False,False


Lo primero será convertir la variable `score_text` a binario. Para ello, mapearemos `Low` $\to$ 0,
`Medium` $\to$ 1, `High` $\to$ 1.

El porqué de esta decisión se explica a detalle en los reportes de Pro-Publica.

In [27]:
post_df["score_text"] = post_df["score_text"].map({"Low": 0, "Medium": 1, "High": 1})
post_df.head()

,age,score_text,priors_count,days_b_screening_arrest,decile_score,is_recid,two_year_recid,c_jail_in,c_jail_out,African-American,Asian,Hispanic,Native American,Other,Greater than 45,Less than 25,Female,M
0,69,0,0,-1.0,1,0,0,2013-08-13 06:03:42,2013-08-14 05:41:20,False,False,False,False,True,True,False,False,False
1,34,0,0,-1.0,3,1,1,2013-01-26 03:45:27,2013-02-05 05:36:53,True,False,False,False,False,False,False,False,False
2,24,0,4,-1.0,4,1,1,2013-04-13 04:58:34,2013-04-14 07:02:04,True,False,False,False,False,False,True,False,False
5,44,0,0,0.0,1,0,0,2013-11-30 04:50:18,2013-12-01 12:28:56,False,False,False,False,True,False,False,False,True
6,41,1,14,-1.0,6,1,1,2014-02-18 05:08:24,2014-02-24 12:18:30,False,False,False,False,False,False,False,False,False


Renombraremos algunas columnas para evitar problemas al utilizar `statsmodels` para ajustar la
regresión logística (no acepta nombres con espacios).

In [28]:
post_df.rename(
    columns={
        "Native American": "Native_American",
        "Greater than 45": "Greater_than_45",

        "Less than 25": "Less_than_25",
        "African-American": "African_American",
    },
    inplace=True,
)


post_df.head()

,age,score_text,priors_count,days_b_screening_arrest,decile_score,is_recid,two_year_recid,c_jail_in,c_jail_out,African_American,Asian,Hispanic,Native_American,Other,Greater_than_45,Less_than_25,Female,M
0,69,0,0,-1.0,1,0,0,2013-08-13 06:03:42,2013-08-14 05:41:20,False,False,False,False,True,True,False,False,False
1,34,0,0,-1.0,3,1,1,2013-01-26 03:45:27,2013-02-05 05:36:53,True,False,False,False,False,False,False,False,False
2,24,0,4,-1.0,4,1,1,2013-04-13 04:58:34,2013-04-14 07:02:04,True,False,False,False,False,False,True,False,False
5,44,0,0,0.0,1,0,0,2013-11-30 04:50:18,2013-12-01 12:28:56,False,False,False,False,True,False,False,False,True
6,41,1,14,-1.0,6,1,1,2014-02-18 05:08:24,2014-02-24 12:18:30,False,False,False,False,False,False,False,False,False


## Regresión Logística

La idea de ajustar una regresión logística entre las predicciones del modelo COMPAS (variable dependiente) y los factores (variables independientes) que consideraremos (delitos anteriores (`priors_count`), comportamiento criminal futuro (`two_year_recid`), raza y grupo etáreo) es justamente poder cuantificar la asociación o efecto de la pertenencia a cierto grupo racial o etáreo sobre la variable dependiente (predicción del modelo COMPAS).

Esto podemos hacerlo mediante el cálculo del **riesgo relativo**. Para esto necesitaremos los coeficientes de la regresión logística.

Partamos definando la fórmula a utilizar en el ajuste para luego ajustar el modelo.



In [29]:
# Definimos la formula.
# Colocamos todos los factores que queremos incluir en el modelo.



formula = "score_text ~ priors_count + two_year_recid + African_American + Asian + Hispanic + Native_American + Other + Greater_than_45 + Less_than_25 + Female + M"

In [30]:
# Instanciamos y ajustamos el modelo

model = sm.formula.glm(
    formula=formula, family=sm.families.Binomial(), data=post_df
).fit()

print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:             score_text   No. Observations:                 6172
Model:                            GLM   Df Residuals:                     6160
Model Family:                Binomial   Df Model:                           11
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -3084.2
Date:                Fri, 16 Aug 2024   Deviance:                       6168.4
Time:                        09:45:57   Pearson chi2:                 6.07e+03
No. Iterations:                     6   Pseudo R-squ. (CS):             0.3128
Covariance Type:            nonrobust                                         
                               coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

### Relative Risk

Recordemos que la regresión logística ajusta una función de la forma:

$
y = \frac{1}{1 + e^{-(\beta_0 + \beta_1X_1 + \beta_2X_2 + \ldots + \beta_pX_p)}}
$

que es equivalente a:

$
y = \frac{e^{\beta_0 + \beta_1X_1 + \beta_2X_2 + \ldots + \beta_pX_p}}{1 + e^{\beta_0 + \beta_1X_1 + \beta_2X_2 + \ldots + \beta_pX_p}}
$

donde en nuestro caso $y$ representa la probabilidad de ser clasificado como alto riesgo de reincidencia.

El riesgo relativo para una variable $X_i$ será:

$RR = \frac{\frac{e^{\beta_0 + \beta_iX_i }}{1 + e^{\beta_0 + \beta_iX_i}}}{\frac{e^{\beta_0}}{1 + e^{\beta_0}}}$


![imagen](https://images.spiceworks.com/wp-content/uploads/2022/04/11040521/46-4-e1715636469361.png)

In [35]:
def relative_risk(coef_0, coef_i):
    group_prob = np.exp(coef_0 + coef_i) / (1 + np.exp(coef_0 + coef_i))
    control = np.exp(coef_0) / (1 + np.exp(coef_0))
    relative_risk = group_prob / control
    return relative_risk

In [32]:
# Riesgo relativo para acusados afro-americanos.
am_rr = relative_risk(-1.5255, 0.4772)
am_rr

np.float64(1.4528254070016209)

Los acusados afroamericanos tienen un 45 % más de probabilidades que los acusados blancos de recibir una puntuación alta si se mantienen controladas la gravedad de su delito, los arrestos anteriores y el comportamiento delictivo futuro.

In [33]:
# Riesgo relativo para menores de 25 años.

young_rr = relative_risk(-1.5255, 1.3084)
young_rr

np.float64(2.496107351371129)

Los acusados jovenes tienen cerca de 2.5 veces la probabilidad que los acusados de mayor edad de recibir una puntuación alta si se mantienen controladas la gravedad de su delito, los arrestos anteriores y el comportamiento delictivo futuro.

In [34]:
# Riesgo relativo para las mujeres

female_rr = relative_risk(-1.5255, 0.2213)
female_rr

np.float64(1.194824380776999)

Las acusadas mujeres tienen un 20% más de probabilidades que los acusados hombres de recibir una puntuación alta si se mantienen controladas la gravedad de su delito, los arrestos anteriores y el comportamiento delictivo futuro.

In [36]:
# Riesgo relativo para los acusados con más de 45 años.

old_rr = relative_risk(-1.5255, -1.3556)
old_rr

np.float64(0.2972006732613585)